 # **Modeling Sale Price using Regression**

Part of CRISP-DM Modelling and Evaluation

## Objectives

* Fit and evaluate a regression model to predict the sale price based on the features in the dataset.
* Perform hyperparameter tuning to optimize the model's performance.
* Identify and analyze the most important features used in model training.

## Inputs

* outputs/datasets/collection/house_prices_records.csv: The raw dataset containing house prices records.

## Outputs

* X_train and y_train: The training set features and target, respectively.
* X_test and y_test: The testing set features and target, respectively.
* Pipeline_regression: The trained machine learning pipeline to predict sale prices.
* Feature_importance_plot: A plot visualizing the importance of each feature in the trained model.
* Model_performance_evaluation.png: A plot evaluating the performance of the trained model on both the training and testing sets.
* Regression_pipeline.pkl: The saved trained regression pipeline model for later use.

# Change working directory

We need to change the working directory from its current folder to its parent folder

* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

Load housing_records.csv into a DataFrame

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv("outputs/dataset/housing_records.csv")
print(data.shape)
data.head()

# **ML Pipeline: Regression**

Create the ML Pipeline

In [ ]:
# Import Pipeline from scikit-learn
from sklearn.pipeline import Pipeline

# Import data cleaning modules from feature_engine
# Data Cleaning
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.outliers   import Winsorizer

# Import feature engineering modules from feature_engine
# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection, DropFeatures
from feature_engine import transformation as vt

# Import feature scaling module from scikit-learn
# Feature Scaling
from sklearn.preprocessing import StandardScaler

# Import feature selection module from scikit-learn
# Feature Selection
from sklearn.feature_selection import SelectFromModel

# Import machine learning algorithms from scikit-learn and xgboost
# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

def pipeline_optimization(model):
    # Define a pipeline with multiple steps
    pipeline_base = Pipeline([
        
        # Drop features 'EnclosedPorch' and 'WoodDeckSF'
        ("DropFeatures", DropFeatures(features_to_drop=['EnclosedPorch', 'WoodDeckSF'])),

        # Impute missing values with 0 for features '2ndFlrSF' and 'MasVnrArea'
        ("ArbitraryNumberImputer",ArbitraryNumberImputer(arbitrary_number=0, variables=['2ndFlrSF','MasVnrArea'])),

        # Impute missing categorical values with 'Unf' for features 'GarageFinish' and 'BsmtFinType1'
        ("CategoricalImputer", CategoricalImputer(imputation_method='missing',fill_value='Unf', variables=['GarageFinish','BsmtFinType1'])),
        
        # Impute missing values with median for features 'BedroomAbvGr', 'GarageYrBlt', and 'LotFrontage'
        ("MedianImputation", MeanMedianImputer(imputation_method='median', variables=['BedroomAbvGr' , 'GarageYrBlt', 'LotFrontage'])),

        # Ordinal encode categorical features 'BsmtExposure', 'BsmtFinType1', 'GarageFinish', and 'KitchenQual'
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary', 
                                                     variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])),

        # Apply log transformation to features '1stFlrSF' and 'GrLivArea'
        ("LogTransformer", vt.LogTransformer(variables=['1stFlrSF','GrLivArea'])),

        # Apply Yeo-Johnson transformation to features 'BsmtUnfSF', 'GarageArea', and 'TotalBsmtSF'
        ("YeoJohnsonTransformer", vt.YeoJohnsonTransformer(variables=['BsmtUnfSF','GarageArea','TotalBsmtSF'])),

        # Apply power transformation to feature 'LotArea'
        ("PowerTransformer", vt.PowerTransformer(variables=['LotArea'])),

        # Winsorize feature 'GrLivArea' to reduce outliers
        ("Winsorizer",Winsorizer(capping_method='iqr', tail='both', fold=1.5, variables=['GrLivArea'])),
                                      
        # Select features based on correlation and variance
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")),

        # Scale features using StandardScaler
        ("scaler", StandardScaler()),

        # Select features based on importance weights from the model
        ("feat_selection", SelectFromModel(model)),

        # Train the machine learning model
        ("model", model),
    ])

    return pipeline_base

In [ ]:
# Import GridSearchCV from scikit-learn
from sklearn.model_selection import GridSearchCV

# Define a class for hyperparameter optimization search
class hyperparameter_optimization_search:

    def __init__(self, models, params):
        # Initialize the class with models and parameters
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        # Iterate over each model and perform GridSearchCV
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            # Create a pipeline for the current model
            model = pipeline_optimization(self.models[key])

            # Get the parameters for the current model
            params = self.params[key]
            # Perform GridSearchCV with the current model and parameters
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            # Store the GridSearchCV object
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        # Define a helper function to create a row for the summary dataframe
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        # Initialize an empty list to store the rows
        rows = []
        # Iterate over each GridSearchCV object
        for k in self.grid_searches:
            # Get the parameters and scores from the GridSearchCV object
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            # Create a row for each parameter setting
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        # Create a dataframe from the rows
        data = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        # Define the columns for the summary dataframe
        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in data.columns if c not in columns]

        # Return the summary dataframe and the GridSearchCV objects
        return data[columns], self.grid_searches

# Split Train and Test Sets

# Import train_test_split from scikit-learn
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    # Features (drop 'SalePrice' column)
    data.drop(['SalePrice'], axis=1),
    # Target variable ('SalePrice' column)
    data['SalePrice'],
    # Test set size (20% of the data)
    test_size=0.2,
    # Random seed for reproducibility
    random_state=0
)

# Print the shapes of the training and testing sets
print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

# Grid Search CV - Sklearn

In [ ]:
# Define a dictionary of machine learning models to search
models_search = {
    # Linear Regression model
    'LinearRegression': LinearRegression(),
    # Decision Tree Regressor model with random state 0 for reproducibility
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    # Random Forest Regressor model with random state 0 for reproducibility
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    # Extra Trees Regressor model with random state 0 for reproducibility
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    # AdaBoost Regressor model with random state 0 for reproducibility
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    # Gradient Boosting Regressor model with random state 0 for reproducibility
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    # XGB Regressor model with random state 0 for reproducibility
    "XGBRegressor": XGBRegressor(random_state=0),
}

# Define a dictionary of hyperparameter search spaces for each model
params_search = {
    # No hyperparameters to search for Linear Regression
    'LinearRegression': {},
    # No hyperparameters to search for Decision Tree Regressor
    "DecisionTreeRegressor": {},
    # No hyperparameters to search for Random Forest Regressor
    "RandomForestRegressor": {},
    # No hyperparameters to search for Extra Trees Regressor
    "ExtraTreesRegressor": {},
    # No hyperparameters to search for AdaBoost Regressor
    "AdaBoostRegressor": {},
    # No hyperparameters to search for Gradient Boosting Regressor
    "GradientBoostingRegressor": {},
    # No hyperparameters to search for XGB Regressor
    "XGBRegressor": {},
}

In [ ]:
# Perform hyperparameter optimization search
search = hyperparameter_optimization_search(
    # Dictionary of machine learning models to search
    models=models_search, 
    # Dictionary of hyperparameter search spaces for each model
    params=params_search
)

# Fit the hyperparameter optimization search to the training data
search.fit(
    # Features of the training set
    X_train, 
    # Target variable of the training set
    y_train, 
    # Evaluation metric for hyperparameter optimization (R-squared)
    scoring='r2', 
    # Number of jobs to run in parallel (-1 means all available cores)
    n_jobs=-1, 
    # Number of folds for cross-validation
    cv=5
)

In [ ]:
# Get the summary of the hyperparameter optimization search
grid_search_summary, grid_search_pipelines = search.score_summary(
    # Sort the results by the mean score of each model
    sort_by='mean_score'
)

# Print the summary of the hyperparameter optimization search
print(grid_search_summary)

In [ ]:
# Get the best-performing model from the search summary
best_model = grid_search_summary.iloc[0, 0]

# Print the best-performing model
print(best_model)

In [ ]:
# Define a dictionary to store the models to search
models_search = {
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
}

# Define a dictionary to store the hyperparameter search space for each model
params_search = {
    "GradientBoostingRegressor": {
        # Hyperparameter search space for GradientBoostingRegressor
        'model__learning_rate': [0.05],  # Search for the best learning rate
        # Other hyperparameters are commented out, but can be uncommented to explore more options
        # 'odel__n_estimators': [50, 75, 100],  # Search for the best number of estimators
        # 'odel__max_depth': [3, 5, 10],  # Search for the best maximum depth
        # 'odel__min_samples_split': [2, 25, 50, 75],  # Search for the best minimum samples to split
        # 'odel__min_samples_leaf': [1, 25, 50, 75],  # Search for the best minimum samples in a leaf
        # 'odel__max_leaf_nodes': [None, 25, 50, 75],  # Search for the best maximum leaf nodes
    }
}


# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
# In a workplace project, you may consider more hyperparameters and spend more time in this step
# https://inria.github.io/scikit-learn-mooc/python_scripts/ensemble_hyperparameters.html

In [ ]:
# Perform hyperparameter optimization search
search = hyperparameter_optimization_search(models=models_search, params=params_search)

# Fit the search object to the training data
search.fit(X_train, y_train, 
           scoring='r2',  # Use R-squared as the evaluation metric
           n_jobs=-1,  # Use all available CPU cores for parallel processing
           cv=5  # Perform 5-fold cross-validation
          )

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

After thoroughly searching various combinations of these 6 parameters, we discovered that the 'learning_rate' parameter is the only one where the default value does not provide the best result when combined with the others. Instead, it works best at 0.05 rather than the default 0.1. All other parameters achieve optimal results using their default values.

# Evaluating Feature Importance

The pipeline is structured to progressively filter and identify the features that offer the highest predictive power. Initially, we decided to exclude two features due to high levels of missing data: 'EnclosedPorch' and 'WoodDeckSF'. Then, the SmartCorrelationSelection step identifies additional features to drop: '1stFlrSF', 'GarageYrBlt', 'GrLivArea', and 'YearRemodAdd'. Lastly, the SelectFromModel step further refines the selection to the most crucial features for predicting the target values.

In [ ]:
import matplotlib.pyplot as plt  # import matplotlib for plotting
import seaborn as sns  # import seaborn for visualization
sns.set_style('whitegrid')  # set seaborn style to whitegrid

# get the number of data cleaning and feature engineering steps in the pipeline
data_cleaning_feat_eng_steps = 10 

# get the column names after data cleaning and feature engineering
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

# get the best features selected by the feature selection step
best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()
].to_list()

# create a DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# print the most important features and plot their importance
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

# plot the feature importance as a bar chart
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()  # show the plot

# Evaluate on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# function to evaluate regression performance on both train and test sets
def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    # evaluate performance on train set
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    # evaluate performance on test set
    regression_evaluation(X_test, y_test, pipeline)


# function to evaluate regression performance on a single dataset
def regression_evaluation(X, y, pipeline):
    # make predictions using the pipeline
    prediction = pipeline.predict(X)
    # print performance metrics
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(3))
    print("\n")


# function to create scatter plots of actual vs predicted values for train and test sets
def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    # make predictions on train and test sets
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    # create a figure with two subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    
    # create scatter plot for train set
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    # create scatter plot for test set
    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    # show the plots
    plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

The performance metrics R2 are excellent for both the training set (0.886) and the test set (0.84), indicating no overfitting. The model meets the business criteria of an R2 score of 0.8 or higher.

## Revised ML Pipeline Optimization

We've streamlined the ML pipeline to focus solely on the most impactful features identified by our analysis: 'OverallQual', 'TotalBsmtSF', '2ndFlrSF', and 'GarageArea'.

In [ ]:
def PipelineOptimization(model):
    optimized_pipeline = Pipeline([

        # Impute missing values for '2ndFlrSF' with 0
        ("ArbitraryNumberImputer", ArbitraryNumberImputer(arbitrary_number=0, variables=['2ndFlrSF'])),

        # Apply Yeo-Johnson transformation to 'GarageArea' and 'TotalBsmtSF'
        ("YeoJohnsonTransformer", YeoJohnsonTransformer(variables=['GarageArea', 'TotalBsmtSF'])),

        # Standardize the features
        ("scaler", StandardScaler()),

        # Add the machine learning model
        ("model", model),
    ])

    return optimized_pipeline

In [ ]:
# the best parameters were determined in the previous notebook

models_search = {
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
}

params_search = {
    "GradientBoostingRegressor": {
        'model__n_estimators': [100],
        'model__max_depth': [3],
        'model__learning_rate': [0.05],
        'model__min_samples_split': [2],
        'model__min_samples_leaf': [1],
        'model__max_leaf_nodes': [None],
    }
}

In [ ]:
# Filter the train and test datasets to include only the best features
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

# Print the shapes of the datasets
print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)

# Display the first 3 rows of the filtered training set
X_train.head(3)

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

In [ ]:
pipeline_regression = grid_search_pipelines[best_model].best_estimator_
pipeline_regression

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline_regression)
regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline_regression)

Saving now the new pipeline like V2. 

# Push files to Repo

In [ ]:
import joblib
import os

version = 'v2'
file_path = f'outputs/ml_pipeline/predict_price/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Train Set

In [ ]:
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train.head()

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test.head()

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

In [ ]:
pipeline_regression

Save the trained regression pipeline model to a file named regression_pipeline.pkl in the specified file_path using Joblib for later use.

In [ ]:
joblib.dump(value=pipeline_regression, filename=f"{file_path}/regression_pipeline.pkl")

Importance Plot

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')

In [ ]:
def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    """
    Function to generate scatter plots for regression model evaluation.

    Parameters:
    X_train (array-like): Training features.
    y_train (array-like): Training target.
    X_test (array-like): Testing features.
    y_test (array-like): Testing target.
    pipeline (object): Trained regression pipeline model.
    alpha_scatter (float, optional): Transparency of scatter plot points. Defaults to 0.5.
    """
    # Predict on training and testing data
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    # Create a figure with two subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

    # Plot training data
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    # Plot testing data
    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    # Save the plot to a file
    plt.savefig(f'{file_path}/model_performance_evaluation.png', bbox_inches='tight')

# Call the function with the trained pipeline
regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline_regression)

# Push files to Repo

* git add .
* git commit -m " " 
* git push 